In [3]:
import pandas as pd
ds = pd.read_csv('Panel Title-data-2024-08-02 17_42_24.csv') 

def extract_details(row):
    try:
        # Parse JSON
        data = json.loads(row['response'])
        
        # Extract line items
        line_items = data.get('data', {}).get('line_items', [])
        
        extracted_data = []
        for item in line_items:
            # Extract relevant fields
            extracted_data.append({
                'claim_data_id': row['claim_data_id'],
                'intimation_number': row['intimation_number'],
                'bill_item_name': item.get('data', {}).get('description', {}).get('value', ''),
                'vitraya_master_category': item.get('data', {}).get('vitraya_master_category', {}).get('value', ''),
                'tariff_item_name': item.get('data', {}).get('tariff', {}).get('tariff_line_item', ''),
                'hospital_charged_amount': item.get('data', {}).get('amount', {}).get('value', 0.0),
                'quantity': item.get('data', {}).get('quantity', {}).get('value', 0.0),
                'irdai_payable': item.get('data', {}).get('tariff', {}).get('irdai_payable', False),
                'admissible_amount': item.get('data', {}).get('tariff', {}).get('admissible_amount', 0.0)
            })
        
        return extracted_data
    except json.JSONDecodeError:
        return []


ds_extracted = ds.apply(extract_details, axis=1)
ds_expanded = pd.DataFrame([item for sublist in ds_extracted for item in sublist])

ds_expanded_sorted = ds_expanded.sort_values(
    by=['claim_data_id', 'intimation_number', 'bill_item_name'],
    ascending=[True, True, True]  # True means ascending order
)

ds_expanded['Line Item Status'] = ds_expanded['bill_item_name'] == ds_expanded['tariff_item_name']

df = ds_expanded.copy()

 #Step 1: Calculate total hospital charged amount and total admissible amount for 'Medicines & Consumables'
med_consumables = df[df['vitraya_master_category'] == 'Medicines & Consumables']
med_consumables_totals = med_consumables.groupby('claim_data_id').agg(
    admissible_amount_total_of_medicine_items=pd.NamedAgg(column='admissible_amount', aggfunc='sum'),
    total_hospital_charged_amount_meds=pd.NamedAgg(column='hospital_charged_amount', aggfunc='sum')
).reset_index()

df['hospital_charged_amount'] = pd.to_numeric(df['hospital_charged_amount'], errors='coerce')
df['admissible_amount'] = pd.to_numeric(df['admissible_amount'], errors='coerce')



# Step 2: Calculate total hospital charged amount and total admissible amount for all items
all_items_totals = df.groupby('claim_data_id').agg(
    total_hospital_charged_amount_all=pd.NamedAgg(column='hospital_charged_amount', aggfunc='sum'),
    total_admissible_amount_all=pd.NamedAgg(column='admissible_amount', aggfunc='sum')
).reset_index()


# Step 3: Merge all totals into a single DataFrame
final_totals = pd.merge(med_consumables_totals, all_items_totals, on='claim_data_id', how='outer')

# Fill NaNs with 0 if necessary
final_totals.fillna(0, inplace=True)

file_path = 'downloads.xlsx'

# Create an Excel writer object
with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
    # Write each DataFrame to a different sheet
    ds_expanded.to_excel(writer, sheet_name='Main', index=False)
    final_totals.to_excel(writer, sheet_name='Totals', index=False)

# Optionally, you can add formatting or other features using the xlsxwriter engine.
print(f'DataFrames have been saved to {file_path}')